In [99]:
import pandas as pd
import json
import requests
import zlib
import requests
import zipfile,io
import tarfile

# 工作描述清理

In [55]:
job_desc = ! cat ./job.json | sed "s/$(echo -ne $'\u2028')//g"

In [58]:
job_desc

['{"custno":"7c01228e-5bc6-479d-9b18-c95e6678a468","jobno":"4371531","job":"國小安親課輔老師","jobcat1":"2016002008","jobcat2":"2016002011","jobcat3":"0","edu":56,"salary_low":990,"salary_high":990,"role":1,"language1":14444,"language2":1111,"language3":1111,"period":2,"major_cat":"0","major_cat2":"0","major_cat3":"0","industry":"1005001009","worktime":"週休二日","role_status":2049,"s2":4,"s3":0,"addr_no":6001005004,"s9":1,"need_emp":1,"need_emp1":2,"startby":2,"exp_jobcat1":"0","exp_jobcat2":"0","exp_jobcat3":"0","description":"※輔導國小課業內容。\\r\\n※中、英語能力尤佳。\\r\\n需有相關課輔經驗者，無經驗者勿試。","others":"1.提供勞健保\\n2.待遇優\\n3.需有耐心.愛心.抗壓性高/配合度佳-無誠者勿試/有安親經歷者尤佳"}',
 '{"custno":"6976d5c5-26a5-467d-8c00-eec6cc569c56","jobno":"9634973","job":"環保專職人員","jobcat1":"2009004003","jobcat2":"2009004008","jobcat3":"2009004001","edu":60,"salary_low":990,"salary_high":990,"role":1,"language1":182222,"language2":14444,"language3":1111,"period":-1,"major_cat":"0","major_cat2":"0","major_cat3":"0","industry":"1002007008","worktime":"週

In [57]:
len(job_desc)

458638

In [59]:
list_correct_parsed_lines = []
list_wrong_parsed_lines = []
for json_line in job_desc:
    try:
        list_correct_parsed_lines.append(json.loads(json_line))           
    except:
#         print(json_line)
        list_wrong_parsed_lines.append(json_line)

In [60]:
len(list_correct_parsed_lines)

458638

In [61]:
len(list_wrong_parsed_lines)

0

In [94]:
df_job_desc = pd.DataFrame(list_correct_parsed_lines)

In [63]:
df_job_desc.to_csv('./20180711_job_info.csv')

In [51]:
df_job_desc = pd.read_csv('./20180711_job_info.csv',
                          usecols=['addr_no','industry','major_cat','major_cat2','major_cat3'],
                          dtype={'addr_no':str,
                                 'industry':str,
                                 'major_cat':str,
                                 'major_cat2':str,
                                 'major_cat3':str
                                })

In [3]:
df_job_desc.columns

Index(['Unnamed: 0', 'addr_no', 'custno', 'description', 'edu', 'exp_jobcat1',
       'exp_jobcat2', 'exp_jobcat3', 'industry', 'job', 'jobcat1', 'jobcat2',
       'jobcat3', 'jobno', 'language1', 'language2', 'language3', 'major_cat',
       'major_cat2', 'major_cat3', 'need_emp', 'need_emp1', 'others', 'period',
       'role', 'role_status', 's2', 's3', 's9', 'salary_high', 'salary_low',
       'startby', 'worktime'],
      dtype='object')

In [118]:
df_district = pd.read_csv('./category/district.csv',
                          error_bad_lines=False,
                          usecols=['類目代碼','類目名稱'],
                         dtype={'類目代碼':str})

In [119]:
df_district

,類目代碼,類目名稱
0,6000000000,地區
1,6001000000,台灣地區
2,6001001000,台北市
3,6001001001,台北市中正區
4,6001001002,台北市大同區
5,6001001003,台北市中山區
6,6001001004,台北市松山區
7,6001001005,台北市大安區
8,6001001006,台北市萬華區
9,6001001007,台北市信義區


In [78]:
df_job_distric = df_job_desc[['addr_no']]

In [86]:
df_job_distric.merge(df_district,right_on='類目代碼',
                                     left_on = 'addr_no')

,addr_no,類目代碼,類目名稱
0,6001005004,6001005004,桃園市楊梅區
1,6001005004,6001005004,桃園市楊梅區
2,6001005004,6001005004,桃園市楊梅區
3,6001005004,6001005004,桃園市楊梅區
4,6001005004,6001005004,桃園市楊梅區
5,6001005004,6001005004,桃園市楊梅區
6,6001005004,6001005004,桃園市楊梅區
7,6001005004,6001005004,桃園市楊梅區
8,6001005004,6001005004,桃園市楊梅區
9,6001005004,6001005004,桃園市楊梅區


In [15]:
df_job_desc.groupby('addr_no')['industry'].value_counts().unstack('addr_no')
# .value_counts()
# 'addr_no','industry'

addr_no,0,10,3,32,4,48,5,56,60,6001001000,...,6008003004,6008003009,6008004000,6008004001,6008005000,6008005002,6008005017,63,7,8
industry,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,1.0,1.0,3.0,NaN,137.0,16.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0
1001001001,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001001002,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001001003,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001001004,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001001005,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001001006,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001002001,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001002003,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_job_desc['industry'] = df_job_desc['industry'].astype('str')

In [30]:
df_industry = pd.read_csv('./category/industry.csv')

In [31]:
df_industry['類目代碼'] = df_industry['類目代碼'].astype('str')

In [35]:
df_ = pd.merge(df_industry,df_job_desc, left_on='類目代碼',right_on='industry')

In [123]:
df_.addr_no = df_.addr_no.astype('str')

In [125]:
df_chinese_addr = pd.merge(df_district,df_,left_on='類目代碼',right_on='addr_no')[['類目名稱_x','類目名稱_y']]

In [131]:
df_chinese_addr.groupby('類目名稱_x')['類目名稱_y'].value_counts().unstack('類目名稱_y').fillna(0)

類目名稱_y,IC設計相關業,不動產經營業,不織布業,中學教育事業,乳品製造業,事務機器製造業,人力仲介代徵,人身保險業,人造纖維製造業,休閒服務業,...,鞋類／布類／服飾品批發業,鞋類／布類／服飾品零售業,食品什貨批發業,食品什貨零售業,食用油品及榖製品製造業,飲料店業,飲料製造業,餐館業,首飾及貴金屬批發業,首飾及貴金屬零售業
類目名稱_x,,,,,,,,,,,,,,,,,,,,,
上海市,25.0,3.0,0.0,1.0,0.0,0.0,165.0,9.0,0.0,2.0,...,3.0,7.0,14.0,21.0,1.0,13.0,2.0,16.0,0.0,2.0
中南美洲,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中歐,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
中美洲,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
亞洲其他,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
其他亞洲,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
加拿大,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
北京市,5.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
南投縣,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
_.to_csv('./20180712_各地區工作類型次數分配表.csv')

In [53]:
df_department = pd.read_csv('./category/department.csv')

In [66]:
df_job_desc.groupby(['major_cat'])['major_cat2'].value_counts().unstack('major_cat2')

major_cat2,0,3001000000,3001001000,3001002000,3001003000,3001004000,3001005000,3002000000,3002001000,3002002000,...,3017004000,3017005000,3017006000,3017007000,3018000000,3018001000,3018002000,3018003000,3018004000,3018005000
major_cat,,,,,,,,,,,,,,,,,,,,,
0,339563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3001000000,108.0,NaN,3.0,NaN,NaN,12.0,1.0,9.0,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,3.0,1.0
3001001000,23.0,1.0,NaN,2.0,5.0,36.0,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3001002000,8.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3001003000,19.0,1.0,4.0,2.0,NaN,2.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
3001004000,355.0,41.0,53.0,7.0,15.0,NaN,47.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,8.0
3001005000,5.0,NaN,1.0,NaN,2.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3002000000,320.0,15.0,NaN,NaN,NaN,3.0,NaN,NaN,9.0,NaN,...,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,1.0,4.0


In [67]:
len(df_job_desc)

459750

In [78]:
df_department.類目代碼.sort_values(ascending=True)

0      3000000000
1      3001000000
2      3001001000
126    3001002000
130    3001003000
134    3001004000
138    3001005000
127    3002000000
3      3002001000
4      3002002000
5      3002003000
6      3002004000
7      3002005000
8      3002006000
9      3002007000
10     3002008000
11     3002009000
131    3003000000
12     3003001000
13     3003002000
14     3003003000
15     3003004000
16     3003005000
17     3003006000
18     3003007000
19     3003008000
20     3003009000
135    3004000000
21     3004001000
22     3004002000
          ...    
110    3014003000
111    3014004000
112    3014005000
113    3014006000
153    3015000000
114    3015001000
115    3015002000
116    3015003000
117    3015004000
118    3015005000
119    3015006000
154    3016000000
120    3016001000
121    3016002000
122    3016003000
123    3016004000
155    3017000000
136    3017001000
124    3017002000
128    3017003000
132    3017004000
140    3017005000
143    3017006000
145    3017007000
156    301

In [79]:
hdd_path = "/media/kuo/bfddc57d-b35d-4995-82c5-a2945d092417/home/hao-chieh/venv/201807_黑客松_104_人力銀行/"

In [80]:
import os

In [81]:
os.listdir(hdd_path)

['testset-click.json']

In [93]:
testset_click = ! cat /media/kuo/bfddc57d-b35d-4995-82c5-a2945d092417/home/hao-chieh/venv/201807_黑客松_104_人力銀行/testset-click.json | sed "s/$(echo -ne $'\u2028')//g"

In [94]:
list_correct_parsed_lines = []
list_wrong_parsed_lines = []
for json_line in testset_click:
    try:
        list_correct_parsed_lines.append(json.loads(json_line))           
    except:
#         print(json_line)
        list_wrong_parsed_lines.append(json_line)

In [95]:
len(list_correct_parsed_lines)

44626

In [97]:
pd.DataFrame(list_correct_parsed_lines)

,id,joblist,querystring
0,1,"[10000096, 10425384, 5025420, 7874246, 1014796...",ro=0&isnew=30&kwop=7&keyword=%E5%8A%A9%E7%90%8...
1,2,"[10000096, 8741137, 10400774, 8606955, 1009176...",ro=0&kwop=7&keyword=%E9%87%91%E8%9E%8D%E4%BF%9...
2,3,"[10000169, 7262152, 8915983, 1054322, 1099425,...",ro=0&jobcat=2011001005%2C2011001002%2C20110010...
3,4,"[10000608, 10001090, 10092793, 10007902, 10113...",ro=0&kwop=7&keyword=%E9%A1%9E%E6%AF%94&order=1...
4,5,"[10000611, 7035365, 8090325, 8492022, 5427407,...",ro=0&isnew=0&kwop=7&keyword=%E5%80%89&area=600...
5,6,"[10000614, 10172390, 10443599, 10429825, 10188...",ro=0&jobcat=2002000000%2C2003000000&kwop=7&key...
6,7,"[10000614, 10172390, 9859553, 10223837, 102251...",ro=0&kwop=7&keyword=%E5%8C%96%E5%AD%B8&area=60...
7,8,"[10000707, 10427490, 8390695, 6674627, 5178016...",ro=0&kwop=7&keyword=%E5%B0%88%E6%AB%83&area=60...
8,9,"[10000727, 10000798, 10194867, 10194874, 10251...",ro=0&kwop=7&keyword=%E6%97%A5%E5%95%86&area=60...
9,10,"[10000942, 10306114, 5395053, 5591806, 5308488...",ro=0&kwop=7&keyword=richemont&order=2&asc=0&mo...


In [107]:
# tarfile.is_tarfile('/tmp/train-action.tar.gz')

# gz = tarfile.open('/tmp/train-action.tar.gz','r:gz')

# len(gz.getnames())

# gz.extractall(hdd_path)

True

# 公司資料

In [23]:
df_company = pd.read_json('./company.json',lines=True,)

In [25]:
df_company = df_company[['name','invoice']]

In [48]:
len(df_company)

340715

In [2]:
res = requests.get('http://www.fia.gov.tw/opendata/bgmopen1.zip')

In [3]:
z = zipfile.ZipFile(io.BytesIO(res.content))
z.extractall('.')

In [15]:
df_invoice_open = pd.read_csv('./BGMOPEN1.csv',usecols=['營業地址','統一編號'])

In [18]:
df_invoice_open = df_invoice_open.dropna()

In [21]:
df_invoice_open['統一編號'] = df_invoice_open['統一編號'].astype('int')

/home/kuo/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_company.merge(df_invoice_open,left_on='invoice',right_on='統一編號')

In [36]:
# _.to_csv('20180711_公司營業地址(only營業中)_.csv')

In [43]:
df_invoice_close = pd.read_csv('./BGMOPEN1X.csv',usecols=['營業地址','統一編號'])

In [44]:
df_invoice_close = df_invoice_closed.dropna()

In [45]:
df_invoice_close['統一編號'] = df_invoice_close['統一編號'].astype('int')

In [ ]:
df_company.merge(df_invoice_close,left_on='invoice',right_on='統一編號')

In [50]:
# _.to_csv('20180711_公司營業地址(only歇業中)_.csv')